此次專案 dataset is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. 現在我們手上有 3份資料：
- Enhanced Twitter Archive
- Additional Data via the Twitter API
- Image Predictions File

### Gather

依照課程提供，我有兩份資料可以直接用 `pandas` 打開
- twitter-archive-enhanced.csv
- image-predictions.tsv

另外還有一份是透過 twitter api 取得資訊，取用的程式我寫在 `tweet.py`，得到一份資料存成 txt 檔，再用 `pandas` 進行讀取。
- tweet_json.txt

這一步我很快的就通過了，沒有碰到太大的困難。

### Access

現在我們手上的三份檔案，其中 Tweet JSON 的資料是用 Twitter archive enhanced table 的 tweet_id 做為條件，透過 twitter api 所取得的資訊，仔細觀察 Twitter archive enhanced table 和 Tweet json table，這兩張 table 其實有許多重複欄位；此外，由於資料來自 Twitter，因此欄位可參考 Twitter document：[Tweet Objects](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html)，接下來我們先用 document 來評估 Tweet JSON 並列出資料的 Quality and tidiness issues.

##### Tweet JSON table

**Quality issues**:

打開資料第一件事，透過 `info` 查看有哪些 _missing data_ ，然後參考 the document of tweet object，我們列出這些欄位並描述如下：

- contributors、coordinates、geo 完全沒有資料，因此我們不考慮這些欄位。
- extended_entities 紀錄附件資訊，缺少這個欄位的資料，我們可以判定該推文並沒有圖片，因此不考慮這些沒有圖片的數據。
- in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str 是同一個群組，當此訊息是回覆推文，會紀錄相關的訊息，那我們只考慮原始推文的資料，所以資料是回覆推文就要刪除。
- possibly_sensitive、possibly_sensitive_appealable 紀錄敏感性資料，我認為這些欄位跟這次的主題沒有太大關係，不用特地保留。
- place 顯示推文地點資訊，但是只有 1筆，因此不保留此欄位。
- quoted_status、quoted_status_id、quoted_status_id_str、quoted_status_permalink 是同一群組，當訊息是引用推文，這些欄位就會紀錄相關訊息，一樣我們不要這些數據。
- retweeted_status: Retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. 因此只要是轉發的數據，我們也不要。

接下來看其它欄位：

- display_text_range 表示文字範圍，不用這個欄位
- entities 原始推文的相關資訊，由於有 image prediction 找到相關圖片預測，因此也不用這個欄位
- favorited 全部都是 False，基本上也不用這個欄位
- id_str 型別錯誤，應為 string object
- is_quote_status: Indicates whether this is a Quoted Tweet. 因此只要是 True 我們不要
- lang 型別錯誤，應為 category datatype
- source 夾雜 html 標籤
- retweeted 都是 False，不用這個欄位
- truncated 都是 False，不用這個欄位
- user 都是同一個帳號 @WeRateDogs，這個欄位也可以移除

透過 Tweet object document，我們對資料有了初步的認識，接下來評估 Twitter archive enhanced 資料比較輕鬆。

##### Twitter archive enhanced table

由於 Twitter archive enhanced table 和 Tweet JSON table 有很多欄位資訊相同，可以預想的到後面清洗資料時可以先合併整理好的 Tweet JSON table 和 Twitter archive enhanced table，留下必要數據。

以下列出此份資料的 issues：

**Quaility issues**

_missing data_
- in_reply_to_status_id、in_reply_to_user_id 欄位資訊在 Tweet JSON 也有，我們可以合併完 table 後再來檢查數據是否為回覆訊息，是的話就要刪除
- retweeted_status_id、retweeted_status_user_id、retweeted_status_timestamp，推測是轉發推文id，可以等 archive table 和 tweet json 資料合併處理後再來檢查。
- expanded_urls，使用 Excel 查看內容發現有許多重複 url，目前不知道目的為何

_其他_
- Erroneous datatypes：timestamp 型別應改為 datetime
- source 和 Tweet JSON 的 source 相同，夾雜 html 標籤語言。

**Tidiness issues**
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果
- rating_numerator, rating_denominator 還隱藏了一個欄位訊息 rating_numerator/rating_denominator 可以得到一個大致的比例

最後評估從 neural network 得到的圖片預測資料集。

##### Image prediction table

**Quality issues**
-  有324筆數據判定不出是哪一種狗的品種

**Tidiness issues**
- 資料存在三種預測結果，裡面隱藏一個欄位即最終結果
- 這份資料可以跟 Twitter archive enhanced table 合併，將預測最好的結果（狗品種）合併過去。

以上我們大致評估完所有的資料

### Clean

##### Tweet json table

- 移除不要的欄位，contributors、coordinates、geo、possibly_sensitive、possibly_sensitive_appealable、place、display_text_range、entities、favorited、retweeted、truncated、user
- 移除 extended_entities 缺失資料
- 刪除 in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str  有值的數據
- 刪除 is_quote_status 為 True 的數據
- 刪除 retweeted_status 有值的數據
- 檢查剩下資料，移除沒有資料的欄位
- 移除 is_quote_status、extended_entities 欄位
- 修正 id_str 型別為 str datatype
- 修正 lang 型別為 category datatype
- 修剪 source 內容不要 html 資訊

#### Twitter archive enhanced table
- 合併 Twitter archive enhanced table 和 Tweet json table。
- 合併後使用 info 檢查資訊，如果欄位沒有任何數據，可直接移除
- 計算 rating_numerator / rating_denominator 並放到新欄位 rating_ratio
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果

### Image prediction table
- 刪除查無品種的數據（一共 324筆）
- 依照預測結果優先順序找出狗品種並放到新開欄位 prediction 
- 與 Twitter archive enhanced table 合併

## Result

After wrangling data, 我們得到的資料如下

In [1]:
import pandas as pd
df = pd.read_csv('twitter_archive_master.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1663 entries, 0 to 1662
Data columns (total 15 columns):
tweet_id          1663 non-null int64
text              1663 non-null object
expanded_urls     1663 non-null object
name              1663 non-null object
doggo             1663 non-null object
floofer           1663 non-null object
pupper            1663 non-null object
puppo             1663 non-null object
favorite_count    1663 non-null int64
lang              1663 non-null object
retweet_count     1663 non-null int64
source            1663 non-null object
created_at        1663 non-null object
rating_ratio      1663 non-null float64
breed             1663 non-null object
dtypes: float64(1), int64(3), object(11)
memory usage: 195.0+ KB


In [2]:
df.head(3)

,tweet_id,text,expanded_urls,name,doggo,floofer,pupper,puppo,favorite_count,lang,retweet_count,source,created_at,rating_ratio,breed
0,892177421306343426,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,None,None,None,None,32434,en,6079,iPhone,2017-08-01 00:17:27,1.3,Chihuahua
1,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,None,None,None,None,24430,en,4023,iPhone,2017-07-31 00:18:03,1.2,Chihuahua
2,891689557279858688,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,None,None,None,None,41088,en,8371,iPhone,2017-07-30 15:58:51,1.3,Labrador_retriever


## Analyze

在分析資料以前，我們針對欄位做資料等級歸納如下，待會兒方便做視覺化：

| Column         | Data Level |
| -------------- | ---------- |
| tweet_id       |  Nominal   |
| text           |  Nominal   |
| expanded_urls  |  Nominal   |
| name           |  Nominal   |
| doggo          |  Nominal   |
| floofer        |  Nominal   |
| pupper         |  Nominal   |
| puppo          |  Nominal   |
| favorite_count |  Discrete  |
| lang           |  Nominal   |
| retweet_count  |  Discrete  |
| source         |  Nominal   |
| rating_ratio   |  Ordinal   |
| breed          |  Nominal   |

接著，我有幾個問題：

Q1. @WeRateDog 比較常評哪幾種狗？

Q2. @WeRateDog 評分最高的狗品種是哪一種？

從資料來說，我們發現 @WeRateDog 評過的狗品種至少有 113種，比較多數的前四名品種依序為：

1) Golden retriever 2) Labrador retriever 3) Pembroke 4) Chihuahua

曾經得到最高評分的是 Pomeranian，結果跟上面不同。

Q3. @WeRateDog 的評分有固定的範圍嗎？

使用箱型圖，我發現評分的範圍很分散，並沒有固定的範圍，感覺起來是隨心所欲的評價。

Q4. @WeRateDog 的評分跟其他人按讚數、轉貼有沒有關係？

使用 `pandas` methond `corr()` 再用 seaborn 的 heatmap 圖。

favorite count and retweet count 很理所當然的就是高度正相關。

不過滿意外的，rating ratio 跟 favorite count and retweet count 沒有很強烈的關係。

從 Q3得知 @WeRateDog 評分範圍很廣所以不再觀察該欄位；
從 Q4 得知 favorite count and retweet count 為高度正相關所以我想取欄位 favorite count 觀察它和欄位 stages、時間的關係。

當我們將資料依 dog stages 分成不同群組，加總 favorite count 再用 Bar chart 顯示發現：

pupper > doggo > puppo > floofer

顯然當推文內容的小狗為 pupper 跟 doggo 這兩種 statages，會受到大家的喜歡。

Q6. 固定時段的推文會不會有更多 favorite count 呢？

我們將資料依小時分群組，再加總所有的按讚數，使用 trend chart 顯示。

BinGo! 從 trend chart 發現有兩段時間（0-2、15-18）是 favorite count 的高峰期，很明顯推文的時間很重要，如果想要推文有更多的轉貼數和點讚數，就要盡量在這兩個時段！

### Insight
1. @WeRateDog 評分沒有特定的模式，從資料看來很隨心所欲
2. 不看評分，觀察大家喜歡哪一類的推文？從資料看來，the stages of dog and time 比較有影響。

In [10]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'wrangle_report.ipynb'])

0